In [1]:
import pandas as pd
from sqlalchemy import create_engine
import time

w_errors=0

In [2]:
wine_start_time = time.time()
wine = "Resources/winemag-data-130k-v2.csv"
wdf = pd.read_csv(wine)
wdf.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
wdf.dtypes

Unnamed: 0                 int64
country                   object
description               object
designation               object
points                     int64
price                    float64
province                  object
region_1                  object
region_2                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
dtype: object

In [4]:
wdf.count()

Unnamed: 0               129971
country                  129908
description              129971
designation               92506
points                   129971
price                    120975
province                 129908
region_1                 108724
region_2                  50511
taster_name              103727
taster_twitter_handle     98758
title                    129971
variety                  129970
winery                   129971
dtype: int64

In [5]:
try:
    wTrans = wdf[["country", "province", "region_1", "winery", "title", "variety", "price", "points", "taster_name", "description", "designation", "taster_twitter_handle"]]
    wTrans.drop_duplicates()
    wTrans.dropna(how='any')
    wTrans.set_index("id", inplace=True)
except:
    w_errors+=1
    
wTrans_count = wTrans.shape[0]
wTrans.head()

,country,province,region_1,winery,title,variety,price,points,taster_name,description,designation,taster_twitter_handle
0,Italy,Sicily & Sardinia,Etna,Nicosia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,NaN,87,Kerin O’Keefe,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,@kerinokeefe
1,Portugal,Douro,NaN,Quinta dos Avidagos,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,15.0,87,Roger Voss,"This is ripe and fruity, a wine that is smooth...",Avidagos,@vossroger
2,US,Oregon,Willamette Valley,Rainstorm,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,14.0,87,Paul Gregutt,"Tart and snappy, the flavors of lime flesh and...",NaN,@paulgwine
3,US,Michigan,Lake Michigan Shore,St. Julian,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,13.0,87,Alexander Peartree,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,NaN
4,US,Oregon,Willamette Valley,Sweet Cheeks,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,65.0,87,Paul Gregutt,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,@paulgwine


In [6]:
wTrans.count()

country                  129908
province                 129908
region_1                 108724
winery                   129971
title                    129971
variety                  129970
price                    120975
points                   129971
taster_name              103727
description              129971
designation               92506
taster_twitter_handle     98758
dtype: int64

In [7]:
#wTrans.groupby(['country']).count() 

In [8]:
opath = "Resources/wine.csv"
wTrans.to_csv(opath)

In [9]:
connection_string = "postgres:postgres@localhost:5432/Wine_etl"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
wTrans.to_sql(name='wine', con=engine, if_exists='replace')
wine_end_time=time.time() - wine_start_time

In [11]:
print(wine_end_time)

35.59701991081238


In [12]:
conn = engine.connect()
conn

In [13]:
W = pd.read_sql("SELECT * FROM wine", conn)
W.count()

index                    129971
country                  129908
province                 129908
region_1                 108724
winery                   129971
title                    129971
variety                  129970
price                    120975
points                   129971
taster_name              103727
description              129971
designation               92506
taster_twitter_handle     98758
dtype: int64

In [14]:
W1=W.dropna(how='any')
W1.count()

index                    52106
country                  52106
province                 52106
region_1                 52106
winery                   52106
title                    52106
variety                  52106
price                    52106
points                   52106
taster_name              52106
description              52106
designation              52106
taster_twitter_handle    52106
dtype: int64

In [15]:
W1

,index,country,province,region_1,winery,title,variety,price,points,taster_name,description,designation,taster_twitter_handle
4,4,US,Oregon,Willamette Valley,Sweet Cheeks,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,65.0,87,Paul Gregutt,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,@paulgwine
5,5,Spain,Northern Spain,Navarra,Tandem,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,15.0,87,Michael Schachner,Blackberry and raspberry aromas show a typical...,Ars In Vitro,@wineschach
6,6,Italy,Sicily & Sardinia,Vittoria,Terre di Giurfo,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,16.0,87,Kerin O’Keefe,"Here's a bright, informal red that opens with ...",Belsito,@kerinokeefe
9,9,France,Alsace,Alsace,Jean-Baptiste Adam,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,27.0,87,Roger Voss,This has great depth of flavor with its fresh ...,Les Natures,@vossroger
10,10,US,California,Napa Valley,Kirkland Signature,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,19.0,87,Virginie Boone,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,@vboone
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129960,129959,France,Alsace,Alsace,Cave de Turckheim,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,57.0,90,Roger Voss,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,@vossroger
129962,129962,Italy,Sicily & Sardinia,Sicilia,Cusumano,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,40.0,90,Kerin O’Keefe,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,@kerinokeefe
129965,129965,France,Alsace,Alsace,Domaine Rieflé-Landmann,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,28.0,90,Roger Voss,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,@vossroger
129968,129968,France,Alsace,Alsace,Domaine Gresser,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,30.0,90,Roger Voss,Well-drained gravel soil gives this wine its c...,Kritt,@vossroger


In [16]:
etl_report=pd.DataFrame({'domain' : ['wine'],
                       'row_counts' : [wTrans_count],
                        'processing_time':[str(round(wine_end_time,2)) + ' seconds'],
                         'error_count':[w_errors]})

In [17]:
etl_report.to_html('etl_wreport.html',index=False)

In [18]:
etl_report

,domain,row_counts,processing_time,error_count
0,wine,129971,35.6 seconds,1


In [19]:
wine

'Resources/winemag-data-130k-v2.csv'

In [35]:
w2=W1.groupby(["country"]).count().reset_index()
w2["country"]

0    Argentina
1    Australia
2       Canada
3       France
4        Italy
5        Spain
6           US
Name: country, dtype: object

In [48]:
w3={}
for i in w2["country"]:
    w3[i]=W1.loc[W1["country"] == i]

    w3[i].to_csv(i)

In [40]:
W1.loc[W1["country"] == i]

,index,country,province,region_1,winery,title,variety,price,points,taster_name,description,designation,taster_twitter_handle
4,4,US,Oregon,Willamette Valley,Sweet Cheeks,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,65.0,87,Paul Gregutt,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,@paulgwine
10,10,US,California,Napa Valley,Kirkland Signature,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,19.0,87,Virginie Boone,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,@vboone
23,23,US,California,Paso Robles,Bianchi,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,22.0,87,Matt Kettmann,This wine from the Geneseo district offers aro...,Signature Selection,@mattkettmann
25,25,US,California,Sonoma Coast,Castello di Amorosa,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,69.0,87,Virginie Boone,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,@vboone
35,35,US,Oregon,McMinnville,Erath,Erath 2010 Hyland Pinot Noir (McMinnville),Pinot Noir,50.0,86,Paul Gregutt,As with many of the Erath 2010 vineyard design...,Hyland,@paulgwine
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129920,129919,US,Washington,Walla Walla Valley (WA),Nicholas Cole Cellars,Nicholas Cole Cellars 2004 Reserve Red (Walla ...,Red Blend,105.0,91,Paul Gregutt,"This ripe, rich, almost decadently thick wine ...",Reserve,@paulgwine
129927,129926,US,Washington,Walla Walla Valley (WA),Reininger,Reininger 2005 Ash Hollow Vineyard Syrah (Wall...,Syrah,41.0,91,Paul Gregutt,This pure Syrah from Reininger's estate vineya...,Ash Hollow Vineyard,@paulgwine
129946,129945,US,California,Santa Ynez Valley,Birichino,Birichino 2013 Jurassic Park Vineyard Old Vine...,Chenin Blanc,20.0,90,Matt Kettmann,Hailing from one of the more popular vineyards...,Jurassic Park Vineyard Old Vines,@mattkettmann
129950,129949,US,California,Napa Valley,Flora Springs,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay,35.0,90,Virginie Boone,There's no bones about the use of oak in this ...,Barrel Fermented,@vboone


In [31]:
us = W1.loc[W1["country"] == "US"]
us

,index,country,province,region_1,winery,title,variety,price,points,taster_name,description,designation,taster_twitter_handle
4,4,US,Oregon,Willamette Valley,Sweet Cheeks,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,65.0,87,Paul Gregutt,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,@paulgwine
10,10,US,California,Napa Valley,Kirkland Signature,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,19.0,87,Virginie Boone,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,@vboone
23,23,US,California,Paso Robles,Bianchi,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,22.0,87,Matt Kettmann,This wine from the Geneseo district offers aro...,Signature Selection,@mattkettmann
25,25,US,California,Sonoma Coast,Castello di Amorosa,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,69.0,87,Virginie Boone,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,@vboone
35,35,US,Oregon,McMinnville,Erath,Erath 2010 Hyland Pinot Noir (McMinnville),Pinot Noir,50.0,86,Paul Gregutt,As with many of the Erath 2010 vineyard design...,Hyland,@paulgwine
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129920,129919,US,Washington,Walla Walla Valley (WA),Nicholas Cole Cellars,Nicholas Cole Cellars 2004 Reserve Red (Walla ...,Red Blend,105.0,91,Paul Gregutt,"This ripe, rich, almost decadently thick wine ...",Reserve,@paulgwine
129927,129926,US,Washington,Walla Walla Valley (WA),Reininger,Reininger 2005 Ash Hollow Vineyard Syrah (Wall...,Syrah,41.0,91,Paul Gregutt,This pure Syrah from Reininger's estate vineya...,Ash Hollow Vineyard,@paulgwine
129946,129945,US,California,Santa Ynez Valley,Birichino,Birichino 2013 Jurassic Park Vineyard Old Vine...,Chenin Blanc,20.0,90,Matt Kettmann,Hailing from one of the more popular vineyards...,Jurassic Park Vineyard Old Vines,@mattkettmann
129950,129949,US,California,Napa Valley,Flora Springs,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay,35.0,90,Virginie Boone,There's no bones about the use of oak in this ...,Barrel Fermented,@vboone
